# Utah Division Of Wildlife Resources Imagery Analyzer
####  *By Bjorn Nicolaisen*

In [1]:
#initialize ee and authenticate if necessary
import ee

ee.Authenticate() # only run this for the initial setup, comment out after authentication
ee.Initialize()

In [2]:
#import the necessary packages

import geemap
import pandas
import ipywidgets as widgets
from ipywidgets import Layout, Button, Box
import altair as alt
from IPython import display
import statistics as stat
import geopandas
from owslib.wfs import WebFeatureService
from requests import Request
import copy

In [16]:
# this section includes all the functions

#this class gets the start and end dates
class get_dates:
    #creates two instance objects by pulling from the field in the IPYwidgets date section
    def __init__(self, sd, ed):
        self.start_date = sd
        self.end_date = ed
        
    @classmethod
    def from_input(cls, UTC):
        #adds the start and end dates from ipywidgets input into the class instance
        #the UTC argument converts the local time entered into UTC
        #timezone is hardcoded to MST
        import pytz
        from datetime import datetime
        from datetime import timedelta
        tz = pytz.timezone("America/Denver")
        
        #set the start date // if the field is blank it will default to the first day of the current year
        input_date = START_DATE.value
        try:
            if input_date == "":
                start_date = tz.localize(datetime(datetime.today().year, 1, 1))
            else:
                start_date = tz.localize(datetime.strptime((input_date), '%Y-%m-%d'))
            
        except:
            print("re-enter using correct date format")

        if UTC == True:
            start_date = start_date.astimezone(pytz.timezone("Etc/UTC"))
            
        #set the end date // if the field is blank it will default to todays date
        input_date = END_DATE.value
        try:
            if input_date == "":
                end_date = tz.localize(datetime.now() + timedelta(days=1)) #add a day because ee end dates are exclusive
            else:
                end_date = tz.localize(datetime.strptime((input_date), '%Y-%m-%d'))
            
        except:
            print("re-enter using correct date format")
        
        if UTC == True:
            end_date = end_date.astimezone(pytz.timezone("Etc/UTC"))
        
        #converts the date to isoformat to easily get gobbled up by earthengine
        return cls(
            start_date.isoformat(),
            end_date.isoformat(),)
    
#this class gets the wildlife crossing from a WFS  
class get_crossings:
    def __init__(self, pt , cr, eept, eecr):
        self.points = pt
        self.bbox = cr
        self.eepoints = eept
        self.eebbox = eecr
        
    @classmethod
    def from_file(cls):
        # get the URL
        url = "https://dservices.arcgis.com/ZzrwjTRez6FJiOq4/arcgis/services/Wildlife_Crossing_Survey/WFSServer"
        
        # initialize
        wfs11 = WebFeatureService(url= url, version='2.0.0')

        # get the layer out of the WFS
        layer = list(wfs11.contents)[-1]
        
        # assemble request parameters
        params = dict(service='WFS', version='2.0.0', request='GetFeature',
              typeNames=layer)
        
        # construct the request url q
        q = Request('GET', url, params=params).prepare().url
        
        # use the request to get points
        points = geopandas.read_file(q)
        
        bufferSize = 1000
        
        #Get the crossing buffers done and then put everything into geopanda geodataframe
        bbox = copy.deepcopy(points)
        bbox["geometry"] = bbox.to_crs("EPSG:6341").buffer(bufferSize, cap_style = 3).to_crs("EPSG:4326")
        
        eepoints = geemap.geopandas_to_ee(points)
        eebbox = geemap.geopandas_to_ee(bbox)
       
        return cls(points, bbox, eepoints, eebbox)
    
#this class stores rois in it
class ROI:
    #This empty class can store roi instances in it. A futute development would be the ability 
    #to store more than one roi in each session. They could be stored in this class. 
    pass

#a bare function
def get_clip_image(ROI):
    #a simple function to set up a clipping function that can easily be mapped to an image collection.
        def clip_image(img):
            return img.clipToCollection(ee.FeatureCollection(ROI))
        return clip_image

def renameMSI(img):
        #this fuction renames sentinel2 bands to make them easier to interpret
        return img.select(
            ["B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B11", "B12", "MSK_SNWPRB", "QA60"],
            [
                "Blue",
                "Green",
                "Red",
                "Red Edge 1",
                "Red Edge 2",
                "Red Edge 3",
                "NIR",
                "Red Edge 4",
                "SWIR1",
                "SWIR2",
                "MSK_SNWPRB",
                "QA60",
            ],
        )

#full of functions to mask snow, clouds
class masking:
    
    #stores all of the functions required to mask snow and cloud.     
    def get_s2_sr_cld_col(ROI, start_date, end_date):
        
        #define a clip function for the roi, to allow it to easily be mapped to an image collection
        def clip_image(img):
            return img.clipToCollection(ee.FeatureCollection(ROI))
        
        # Import and filter S2 SR.
        s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
            .filterBounds(ROI)
            .filterDate(start_date, end_date)
            .map(clip_image))

        # Import and filter s2cloudless.
        s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
            .filterBounds(ROI)
            .filterDate(start_date, end_date)
            .map(clip_image))

        # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
        return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
            'primary': s2_sr_col,
            'secondary': s2_cloudless_col,
            'condition': ee.Filter.equals(**{
                'leftField': 'system:index',
                'rightField': 'system:index'
            })
        }))
    
    #nested functions below that can be called tto set certain aguments and 
    #output a function with just one argument: img
    
    def get_add_snow_mask(SNW_PRB_THRESH):
        def add_snow_mask(img):
            #just run the threshold
            is_snow = img.select('MSK_SNWPRB').gt(SNW_PRB_THRESH).rename('snow')

            return img.addBands(ee.Image([is_snow]))
        return add_snow_mask
    
    def get_add_cld_mask(BUFFER, CLD_PRB_THRESH):
       
        # Condition s2cloudless by the probability threshold value
        
        def add_cld_mask(img):
            
            def add_cloud_bands(img):
                # Get s2cloudless image, subset the probability band.
                cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

                # Condition s2cloudless by the probability threshold value.
                is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

                # Add the cloud probability layer and cloud mask as image bands.
                return img.addBands(ee.Image([cld_prb, is_cloud]))
            
            # Add cloud component bands.
            img_cloud = add_cloud_bands(img)

            #set cloud as value 1, else 0.
            is_cld = img_cloud.select('clouds').gt(0)

            # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
            is_cld = (is_cld.focal_min(2).focal_max(BUFFER*2/20)
                .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
                .rename('cloudmask'))

            # Add the final cloud mask to the image.
            return img_cloud.addBands(is_cld)
        return add_cld_mask
    
    
    #apply the cloud mask to the image (actually mask out the clouds)
    def apply_cld_mask(img):
        # Subset the cloudmask band and invert it so clouds are 0, else 1.
        not_cld = img.select('cloudmask').Not()

        # Subset reflectance bands and update their masks, return the result.
        return img.updateMask(not_cld)
    
    def apply_snow_mask(img):
        # Subset the snowmask band and invert it so clouds are 0, else 1.
        not_snow = img.select('snow').Not()

        # Subset reflectance bands and update their masks, return the result.
        return img.updateMask(not_snow)

#(vegetation indicies!!)
class index:
    def evi (image):
        # a function to add the evi to an image after the bands have been renamed
        evi = image.expression('evi = 2.5 * (((NIR/10000) - (RED/10000)) / (((NIR/10000) + 6 * (RED/10000) - 7.5 * (BLUE/10000)) + 1))',\
                               {'NIR': image.select('NIR'),'RED': image.select('Red'),'BLUE': image.select('Blue')})\

        # add the band 
        image1 = image.addBands(evi)
        return image1
    
    def ndpi (image):
        # a function to add the NDPI to an image after the bands have been renamed
        # see reference 4
        ndpi = image.expression('ndpi = (NIR -(.74 * RED + (.26 * SWIR)))/(NIR +(.74 * RED + (.26 * SWIR)))',\
            {'NIR': image.select('Red Edge 4'),'RED': image.select('Red'),'SWIR': image.select('SWIR2')})\

        #add the band 
        image1 = image.addBands(ndpi)
        return image1



#Reduction
class reduction:
    
    # functions for recucing images to sincle values and appending them to a dataframe
    def get_reduce_for_roi(ROI):
        # get the reducer function for a given roi, so that it can be mapped to an imagecollection
        def reduce(image):
            mean = image.reduceRegion(geometry = ROI, reducer = ee.Reducer.mean(), maxPixels=1e13)
            return ee.Feature(ROI, mean).set({'millis': image.date().millis()})
        return reduce
    
    def fc_to_dict(fc):
        # convert the fc to a dictionary
        prop_names = fc.first().propertyNames()
        prop_lists = fc.reduceColumns(
            reducer = ee.Reducer.toList().repeat(prop_names.size()),
            selectors = prop_names).get('list')

        return ee.Dictionary.fromLists(prop_names, prop_lists)
    
    def add_date_info(df):
        # add date info based on the "millis" attribute
        df['Timestamp'] = pandas.to_datetime(df['millis'], unit='ms')
        df['Year'] = pandas.DatetimeIndex(df['Timestamp']).year
        df['Month'] = pandas.DatetimeIndex(df['Timestamp']).month
        df['Day'] = pandas.DatetimeIndex(df['Timestamp']).day
        df['DOY'] = pandas.DatetimeIndex(df['Timestamp']).dayofyear
        return df

class ipy:
    # Holds variables that define ipywidget style
    
    # style of the Ipywidgets
    style = {'description_width': 'initial'}
    title = widgets.Text(
        description='Title:',
        value='Landsat Timelapse',
        width=200,
        style=style
    )



    lyt_s = Layout(width='100%')
    
    lyt_box = Layout(grid_template_columns='50% 50%',
                    align_items='stretch',
                    width='100%')

class outputs:
    # this class contains functions to create the outputs needed
    # they build upon lower level functions
    def make_chart(DF, name_of_index):
        # make an altair timeseries chart
        highlight = alt.selection(
            type='single', on ='mouseover', fields=['Year'], nearest=True)

        base = alt.Chart(DF).encode(
            x=alt.X('DOY', title= "DOY (Day Of Year)", scale=alt.Scale(domain=[0, 365], clamp=True)),
            y=alt.Y(name_of_index + '_MA', title= name_of_index.upper() + ' Moving Average' , scale=alt.Scale(domain=[0.0, 0.9])),
            color=alt.Color('Year:O', scale=alt.Scale(scheme='viridis'))).properties(title=name_of_index.upper() + ' Timeseries')

        points = base.mark_circle().encode(
            opacity=alt.value(0),
            tooltip=[
                alt.Tooltip('Year', title='Year'),
                alt.Tooltip('Timestamp', title='Date'),
                alt.Tooltip(name_of_index+'_MA', title=name_of_index)
            ]).add_selection(highlight)

        lines = base.mark_line().encode(
            size=alt.condition(~highlight, alt.value(3), alt.value(5)))

        chart = (points + lines).properties(width=600, height=350).interactive().configure(background='#E8E8E8')
        return chart

    def make_summary_chart(DF, name_of_index):
        # make box plot for each year
        ch = alt.Chart(DF).mark_boxplot().encode(
            x='Year:O',
            y=alt.Y(name_of_index+'_MA', title = name_of_index.upper() + ' Moving Average'))

        return ch

    def summary_df(DF, name_of_index):
        # returns the max veg_index value for each year and the date of the max
        
        # create groups for each year
        groups = DF.groupby("Year")

        # initialize some lists
        year = []
        max_val = []
        mean_val = []
        timestamp_list = []
        DOY_list = []
        # loop through each of the groups
        for name, group in groups:
            year.append(name)   #list of years
            vegind = (group[name_of_index+'_MA']).tolist() #grab the veg-index values as a list
            max_veg = max(vegind)  #find the max veg-index value
            mean_veg = stat.mean(vegind)  #find the mean veg-index value
            index_of_max = vegind.index(max_veg) #get the index of the max vaue
            timestamp = (group['Timestamp']).tolist()[index_of_max] #get the timestamp using the index
            DOY = (group['DOY']).tolist()[index_of_max] #get the DOY using the index

            # append all the values to make the df
            max_val.append(round(max_veg, 4))
            mean_val.append(round(mean_veg, 4))
            timestamp_list.append(str(timestamp.month) + '-' + str(timestamp.day))
            DOY_list.append(DOY)

        return pandas.DataFrame(list(zip(year, max_val, mean_val, timestamp_list, DOY_list)),\
            columns =['Year', 'Maximum ' + name_of_index.upper(), 'Mean ' + name_of_index.upper(), 'Date of Max', 'DOY of Max'])

def mosaicByDate(imcol):
    # takes an ImageCollection and returns an ImageCollection, with mosaiced images for each day
    
    # create a list of images
    imlist = imcol.toList(imcol.size())
    
    # make a list of unique dates
    unique_dates = imlist.map(lambda im: ee.Image(im).date().format("YYYY-MM-dd")).distinct()
    
    # function to mosaic images with same date
    def mosaic_dates(d):
        d = ee.Date(d)
        
        # ee end dates are exclusive(that is why we advance one day)                    
        im = imcol\
        .filterDate(d, d.advance(1, "day"))\
        .mosaic()
        
        
        
        return im.set(
        "system:time_start", d.millis(), 
        "system:id", d.format("YYYY-MM-dd"))
        
    
    mosaic_imlist = unique_dates.map(mosaic_dates)             
    
    return ee.ImageCollection(mosaic_imlist)

## ROI Selection Map

In [4]:
# set up a map to grab rois

crossings = get_crossings.from_file()
Map = geemap.Map(center=(39.5, -111.6), zoom=7)

Map.addLayer(crossings.eepoints)
Map

/Users/bjornnicolaisen/opt/miniconda3/envs/gee/lib/python3.8/site-packages/geopandas/geodataframe.py:422: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


Map(center=[39.5, -111.6], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childre…

### Set up the Analysis
1) *Select an ROI using any of the shape creation tools on the left edge of the MAP*

2) *Select start and end dates below in* ***YYYY-mm-dd*** *format*

In [5]:
# start and end date arguments

START_DATE = widgets.Text(placeholder='Leave Blank To Start at 1st DOY', description='Start Date:',disabled=False)

END_DATE = widgets.Text(placeholder='Leave Blank To End Today', description='End Date:', disabled=False)

box = widgets.GridBox(children = [START_DATE, END_DATE], layout = ipy.lyt_box)
box

GridBox(children=(Text(value='', description='Start Date:', placeholder='Leave Blank To Start at 1st DOY'), Te…

### Adjust the arguments for the Vegetation Index Plot
1) *Adjust the Cloud and snow probability threshold values to adjust the masking (values represent percent probability of cover)*

2) *The Buffer value can be increased to capture the edges of clouds*

3) *Select an index to display on the plot (evi and ndpi currently)*

4) *Adjust the moving average window size to smooth out the data*


In [6]:
# Ipywidgets for initializing the plot arguments

# submit button
submit_plot = widgets.Button(
    description='Index Plot',
    button_style='primary',
    tooltip='click to create veg-index plot',
    style=ipy.style
)

output_plot = widgets.Output()

# tuning the cloud masking
CLD_PRB_THRESH = widgets.IntSlider(description='Cloud Probability Threshold', value=50, min=1, max=100, style=ipy.style, layout=ipy.lyt_s)
BUFFER = widgets.IntSlider(description='Cloud Buffer (m)', value=50, min=0, max=100, style=ipy.style, layout=ipy.lyt_s)
SNW_PRB_THRESH = widgets.IntSlider(description='Snow Probability Threshold', value=50, min=1, max=101, style=ipy.style, layout=ipy.lyt_s)
moving_avg_win = widgets.IntSlider(description='Moving Average Window', value=4, min=1, max=15, style=ipy.style, layout=ipy.lyt_s)

INDEX =  widgets.Dropdown(
    description='Select Index to Plot:',
    options=['evi', 'ndpi'],
    value='ndpi',
    style=ipy.style
)

# put all the widgets in a box and display it
box1 = widgets.GridBox(children = [CLD_PRB_THRESH, BUFFER, SNW_PRB_THRESH, moving_avg_win, INDEX, submit_plot], layout = ipy.lyt_box)
box1

GridBox(children=(IntSlider(value=50, description='Cloud Probability Threshold', layout=Layout(width='100%'), …

### Adjust the arguments for the Timeseries Imagery Inspector
1) *Choose a set of bands to display as RGB*

2) *The reflectance values mapped to the dispay can also be adjusted. Increase the upper value to view bright surfaces.*

In [7]:
# Ipywidgets for initializing the TS inspector

# submit button
submit_ts = widgets.Button(
    description='TS Inspector',
    button_style='primary',
    tooltip='click to create splitscreen timeseries inspector',
    style=ipy.style
)

# output widget
output_ts = widgets.Output()

# band selection
bands = widgets.Dropdown(
    description='Select RGB Combo:',
    options=['Red/Green/Blue', 'NIR/Red/Green', 'SWIR2/Red Edge 4/Red', 'evi'],
    value='NIR/Red/Green',
    style=ipy.style
)

# reflectance scaling
ref = widgets.IntRangeSlider(
    value=[0, 5000],
    min=0,
    max=10000,
    step=100,
    description='Scale Ref:',)
   
# put all the widgets in a box and display it
box2 = widgets.GridBox(children = [bands, ref, submit_ts], layout = ipy.lyt_box)
box2

GridBox(children=(Dropdown(description='Select RGB Combo:', index=1, options=('Red/Green/Blue', 'NIR/Red/Green…

In [8]:
# function to create and display TS inspector

def submit_ts_clicked(b):
    with output_ts:
        output_ts.clear_output()
        
        # Grab the ROI and dates
        roi = ROI()
        roi.a = Map.draw_last_feature.geometry()
        dates = get_dates.from_input(UTC = True)
        
        # initialize the clip_image function with the roi
        clip_image = get_clip_image(ROI = roi.a)

        # create a function to grab the proper image collection
        def im_col_for_inspector(ROI):
            s2 = ee.ImageCollection('COPERNICUS/S2_SR')\
                    .filterBounds(ROI)\
                    .filterDate(dates.start_date, dates.end_date)\
                    .map(clip_image)\
                    .map(renameMSI)\
                    .map(index.evi)

            return s2

        # get an image colletion with a clipped image for each day
        images = mosaicByDate(im_col_for_inspector(ROI = roi.a))
        
        # get a list of dates for every image in the above collection
        datelist = ee.List(images \
            .aggregate_array('system:time_start')) \
            .map(lambda time_start: 
                 ee.Date(time_start).format('Y-MM-dd')
            ) .getInfo()

        
        # set the band vis parameters
        if bands.value == 'evi':
            vis = {
            'min': 0,
            'max': 1,
            'bands': bands.value.split('/'),
            'palette' : ['f7fcfd', 'e5f5f9', 'ccece6', '99d8c9', '66c2a4', '41ae76', '238b45', '006d2c', '00441b']}
        else: 
            vis = {
                'min': ref.value[0],
                'max': ref.value[1],
                'gamma': [1, 1, 1],
                'bands': bands.value.split('/')}
        
        # initialize a map
        Map1 = geemap.Map()
        
        # Create a geemap timeseries inspector with the images, vis, and datelist.
        Map1.ts_inspector(left_ts=images, right_ts=images, left_names=datelist, right_names=datelist, left_vis=vis, right_vis=vis)
        
        # center the map on the roi
        Map1.centerObject(roi.a, zoom=10)
        
        # output display the map
        display.display(Map1)
        
# function to handle clicks       
submit_ts.on_click(submit_ts_clicked)

# outputs    
output_ts



Output()

In [9]:
# function to create and display plots
def submit_plot_clicked(b):
    with output_plot:
        output_plot.clear_output()
        
        
        # Grab the ROI and dates
        roi = ROI()
        roi.a = Map.draw_last_feature.geometry()
        dates = get_dates.from_input(UTC = True)

        # reduce all the functions down to one variable (img) to get them to map well
        add_snow_mask = masking.get_add_snow_mask(SNW_PRB_THRESH = SNW_PRB_THRESH.value)
        add_cld_mask = masking.get_add_cld_mask(BUFFER = BUFFER.value, CLD_PRB_THRESH = CLD_PRB_THRESH.value)
        reduce = reduction.get_reduce_for_roi(ROI = roi.a)
        
        # create a function to grab the proper image collection
        def im_col_for_plot(ROI):  

            s2 = masking.get_s2_sr_cld_col(ROI = ROI, start_date = dates.start_date, end_date = dates.end_date)\
                    .map(add_cld_mask)\
                    .map(masking.apply_cld_mask)\
                    .map(add_snow_mask)\
                    .map(masking.apply_snow_mask)\
                    .map(renameMSI)\
                    .map(index.evi)\
                    .map(index.ndpi)\
                    .select(['evi', 'ndpi'])
            
            return s2
        
        # define a function using the reduce function above to create a dataframe from the image collection
        def make_df(col):

            import pandas
            mean = ee.FeatureCollection(col.map(reduce)).filter(
                ee.Filter.notNull(col.first().bandNames()))

            # use the function from above to try to get to dictionary
            mean_dict = reduction.fc_to_dict(mean).getInfo()

            # convert to pandas dataframe and plot
            mean_df = pandas.DataFrame(mean_dict) 

            mean_df = reduction.add_date_info(mean_df)

            mean_df['evi_MA'] = round(mean_df['evi'].rolling(window=moving_avg_win.value, min_periods=0, center=True).mean(), 5)
            mean_df['ndpi_MA'] = round(mean_df['ndpi'].rolling(window=moving_avg_win.value, min_periods=0, center=True).mean(), 5)

            return mean_df
        
        # run the functions above to get the dataframe
        df = make_df(im_col_for_plot(roi.a))
        
        # create each of the outputs using the df
        chart = outputs.make_chart(DF = df, name_of_index = INDEX.value)
        summary_chart = outputs.make_summary_chart(DF = df, name_of_index = INDEX.value)
        summary_dataf = outputs.summary_df(DF = df, name_of_index = INDEX.value)
        
        # display the outputs
        display.display(chart, summary_chart, summary_dataf)


# Function to handle clicks      
submit_plot.on_click(submit_plot_clicked)

# output
output_plot



Output()

### References

1) Jdbcode. (2021). Sentinel-2 cloud masking with S2CLOUDLESS | Google Earth Engine. Retrieved April 17, 2021, from https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless

2) Jdbcode. (2021). Time series visualization with altair | google earth engine. Retrieved April 17, 2021, from https://developers.google.com/earth-engine/tutorials/community/time-series-visualization-with-altair

3) Wu, Q., (2020). geemap: A Python package for interactive mapping with Google Earth Engine. The Journal of Open Source Software, 5(51), 2305. https://doi.org/10.21105/joss.02305

4) Cong Wang, Jin Chen, Jin Wu, Yanhong Tang, Peijun Shi, T. Andrew Black, Kai Zhu,
        A snow-free vegetation index for improved monitoring of vegetation spring green-up date in deciduous
        ecosystems,
        Remote Sensing of Environment,
        Volume 196,
        2017,
        Pages 1-12,
        ISSN 0034-4257,
        https://doi.org/10.1016/j.rse.2017.04.031.
        (https://www.sciencedirect.com/science/article/pii/S0034425717301906)
        
5) VanderPlas, J., Granger, B., Heer, J., Moritz, D., Wongsuphasawat, K., Satyanarayan, A., … Sievert, S. (2018). Altair: Interactive statistical visualizations for python. Journal of Open Source Software, 3(32), 1057.

6) McKinney, W., & others. (2010). Data structures for statistical computing in python. In Proceedings of the 9th Python in Science Conference (Vol. 445, pp. 51–56).

7) Jordahl, K. (2014). GeoPandas: Python tools for geographic data. URL: Https://Github. Com/Geopandas/Geopandas.

8) Kluyver, T., Ragan-Kelley, B., Fernando P&#x27;erez, Granger, B., Bussonnier, M., Frederic, J., … Willing, C. (2016). Jupyter Notebooks – a publishing format for reproducible computational workflows. In F. Loizides & B. Schmidt (Eds.), Positioning and Power in Academic Publishing: Players, Agents and Agendas (pp. 87–90).
        